In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress

In [2]:
#Read in csv files

napaData_df = pd.read_csv('resources/napa_weather.csv') # replcae with correct file path
marinData_df = pd.read_csv('resources/marin_weather.csv') # replcae with correct file path
placerData_df = pd.read_csv('resources/placer_weather.csv') # replcae with correct file path
santaCruzData_df = pd.read_csv('resources/santa_cruz_weather.csv') # replcae with correct file path
sonomaData_df = pd.read_csv('resources/santa_cruz_weather.csv') # replcae with correct file path
caliWine_df = pd.read_csv('resources/Californa_Wine_Production_1980_2020.csv')  # replcae with correct file path

In [3]:
# Clean caliWine df to only include top five price per ton counties 

#reduce dataframe to only include relevant columns
cleaned_wine_df = caliWine_df[['Year', 'County',
       'HarvestedAcres', 'Yield(Unit/Acre)', 'Production',
       'Price(Dollars/Unit)', 'Unit', 'Value(Dollars)']].copy()

cleaned_wine_df.head()




,Year,County,HarvestedAcres,Yield(Unit/Acre),Production,Price(Dollars/Unit),Unit,Value(Dollars)
0,2020,Alameda,2530.0,5.14,13000.0,1497.69,Tons,19470000
1,2020,Amador,5360.0,2.31,12400.0,1318.31,Tons,16347000
2,2020,Calaveras,579.0,3.06,1770.0,1325.99,Tons,2347000
3,2020,Colusa,747.0,6.02,4500.0,684.67,Tons,3081000
4,2020,ContraCosta,1940.0,4.69,9090.0,751.27,Tons,6829000


In [4]:
#find top five price per unit counties

wine_2020 = cleaned_wine_df[cleaned_wine_df['Year'] == 2020]

sorted_wine_df = wine_2020.sort_values(by = 'Price(Dollars/Unit)', ascending = False)

sorted_wine_df.head()

,Year,County,HarvestedAcres,Yield(Unit/Acre),Production,Price(Dollars/Unit),Unit,Value(Dollars)
15,2020,Napa,45200.0,2.20,99300.0,4646.48,Tons,461395000
11,2020,Marin,195.0,1.33,260.0,3607.69,Tons,938000
28,2020,SantaCruz,670.0,2.39,1600.0,3008.75,Tons,4814000
31,2020,Sonoma,56800.0,2.61,148000.0,2375.08,Tons,351512000
17,2020,Placer,315.0,2.79,879.0,2235.49,Tons,1965000


In [5]:
# Make list of top five producing counties

county_list = []

county_list = sorted_wine_df['County'][:5].tolist()

print(county_list)


['Napa', 'Marin', 'SantaCruz', 'Sonoma', 'Placer']


In [6]:
#Reduce wine dataframe to only include 2020 top five producing counties for all years of data

top_five_df = cleaned_wine_df[cleaned_wine_df['County'].isin(county_list)]

#Reindex new top_five_df

top_five_df = top_five_df.reset_index(drop = True)

#Save dataframe to csv and save in the resources folder

top_five_df.to_csv('resources/top_five_counties.csv', index = False)

top_five_df.head()

,Year,County,HarvestedAcres,Yield(Unit/Acre),Production,Price(Dollars/Unit),Unit,Value(Dollars)
0,2020,Marin,195.0,1.33,260.0,3607.69,Tons,938000
1,2020,Napa,45200.0,2.20,99300.0,4646.48,Tons,461395000
2,2020,Placer,315.0,2.79,879.0,2235.49,Tons,1965000
3,2020,SantaCruz,670.0,2.39,1600.0,3008.75,Tons,4814000
4,2020,Sonoma,56800.0,2.61,148000.0,2375.08,Tons,351512000


In [7]:
#Merge all weather data into one dataframe and split year

#insert column for county name
marinData_df.insert(1, 'Name', 'Marin')
napaData_df.insert(1, 'Name', 'Napa')
placerData_df.insert(1, 'Name', 'Placer')
santaCruzData_df.insert(1, 'Name', 'SantaCruz')
sonomaData_df.insert(1, 'Name', 'Sonoma')

#merge all dataframes

merged_weather_df = pd.concat([marinData_df, napaData_df, placerData_df, santaCruzData_df, sonomaData_df], axis = 0)

# split the month column into months and years column
merged_weather_df[['month', 'year']] = merged_weather_df['Month'].str.split(',', expand=True)


merged_weather_df.head()

,Month,Name,Avg. Temp,Min. Temp,Max. Temp,Precip.,month,year
0,"December, 2020",Marin,51.5,43.2,59.7,2.83,December,2020
1,"November, 2020",Marin,53.5,44.2,62.8,1.54,November,2020
2,"October, 2020",Marin,66.2,54.4,78.0,0.01,October,2020
3,"September, 2020",Marin,66.8,56.7,77.0,0.00,September,2020
4,"August, 2020",Marin,66.7,56.4,77.1,0.25,August,2020


In [8]:
#drop 'Month' column and save new dataframe

merged_weather_df = merged_weather_df.drop('Month', axis=1)

#Save dataframe to csv and save in the resources folder

merged_weather_df.to_csv('resources/merged_weather.csv', index = False)

In [12]:
#Define a function to create aggregate dataframes for yearly weather metrics for each county and then merge them into one large dataframe

weather_dfs = [marinData_df, napaData_df, placerData_df, santaCruzData_df, sonomaData_df]
yearly_weather_data_df = pd.DataFrame()

for county_df in weather_dfs:

    # split the month column into months and years columns
    county_df[['month', 'year']] = county_df['Month'].str.split(',', expand=True)

    #drop 'Month' column and save new dataframe
    county_df = county_df.drop('Month', axis=1)

    #use groupby statements to find yearly metrics
    county_groupby = county_df.groupby(['year', 'Name'])

    yearly_aggregate = county_groupby.agg({'Min. Temp': 'min', 'Max. Temp': 'max', 'Precip.': 'sum'})

    #create a data frame of the groupby statements and reset the index to include Name and Year again
    yearly_county_df = pd.DataFrame(yearly_aggregate).reset_index()
    
    #Add yearly temp range
    yearly_county_df['Temp Range'] = yearly_county_df['Max. Temp'] - yearly_county_df['Min. Temp']

    #merge yearly_county_df to larger all counties dataframe to save all info in same df
    yearly_weather_data_df = pd.concat([yearly_weather_data_df, yearly_county_df])




yearly_county_df

,year,Name,Min. Temp,Max. Temp,Precip.,Temp Range
0,1980,Sonoma,39.2,75.9,31.29,36.7
1,1981,Sonoma,41.5,80.6,35.89,39.1
2,1982,Sonoma,34.9,77.6,54.26,42.7
3,1983,Sonoma,40.4,80.3,68.71,39.9
4,1984,Sonoma,37.9,84.4,22.82,46.5
5,1985,Sonoma,36.3,79.1,23.63,42.8
6,1986,Sonoma,38.4,75.1,36.75,36.7
7,1987,Sonoma,37.7,76.6,27.66,38.9
8,1988,Sonoma,37.7,78.8,19.66,41.1
9,1989,Sonoma,35.7,78.4,17.17,42.7
